In [0]:

# ====================
# 1. Set up credentials
# ====================
spark.conf.set(
    "fs.azure.account.key.mmixstorage.blob.core.windows.net",
    "UZTHs33FPYTUvC9G51zk+DQQp/FWf31YOteoW+dEnKuprRgxvk53yS+IpEiLn1062IBpOyoKaXp4+AStRcA1Cw=="
)

# ====================
# 2. Get notebook parameters
# ====================
dbutils.widgets.text("input_path", "")
dbutils.widgets.text("output_path", "")
dbutils.widgets.text("group_col", "")
dbutils.widgets.text("date_col", "")
dbutils.widgets.text("target_col", "")
dbutils.widgets.text("transformations", "{}")

import json
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DateType

def sql_safe(colname):
    return f"`{colname}`"

# Load parameters
input_path = dbutils.widgets.get("input_path")
output_path = dbutils.widgets.get("output_path")
group_col = dbutils.widgets.get("group_col")
date_col = dbutils.widgets.get("date_col")
target_col = dbutils.widgets.get("target_col")
transformations = json.loads(dbutils.widgets.get("transformations"))

print("Transformations:")
print(json.dumps(transformations, indent=2))

# ====================
# 3. Read input CSV
# ====================
df = spark.read.option("header", True).option("inferSchema", True).csv(
    f"wasbs://mmix-blob-storage@mmixstorage.blob.core.windows.net/{input_path}"
)

# Convert date column to DateType
df = df.withColumn(date_col, F.to_timestamp(F.col(date_col), "M/d/yyyy H:mm"))

# ====================
# 4. Apply adstock + saturation
# ====================
for col, trans in transformations.items():
    lags = int(trans.get("lags", 0))
    decay = float(trans.get("decay", 0.0))

    transformed_col = f"{col}_transformed"

    # ----- Apply Adstock -----
    if lags > 0 and decay > 0:
        for i in range(1, lags + 1):
            df = df.withColumn(f"{col}_lag{i}", F.lag(col, i).over(Window.partitionBy(group_col).orderBy(date_col)))

        expr = f"COALESCE({sql_safe(col)}, 0)"
        for i in range(1, lags + 1):
            expr += f" + POWER({decay}, {i}) * COALESCE({sql_safe(col + '_lag' + str(i))}, 0)"

        df = df.withColumn(transformed_col, F.expr(f"({expr})"))
        df = df.drop(*[f"{col}_lag{i}" for i in range(1, lags + 1)])
    else:
        df = df.withColumn(transformed_col, F.col(col))

    # ----- Apply Saturation -----
    saturation = trans.get("saturation")
    if saturation:
        mode = saturation.get("mode", "power")
        k = float(saturation.get("k", 1))

        if mode == "log":
            df = df.withColumn(transformed_col, F.pow(F.log1p(F.col(transformed_col)), k))
        else:
            df = df.withColumn(transformed_col, F.pow(F.col(transformed_col), k))

    # ----- Overwrite original column -----
    # df = df.drop(col).withColumnRenamed(transformed_col, col)

# ====================
# 5. Write transformed CSV to output path
# ====================
df.show()
df.printSchema()

df.write.mode("overwrite").option("header", True).csv(
    f"wasbs://mmix-blob-storage@mmixstorage.blob.core.windows.net/{output_path}"
)


Transformations:
{}


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7518137628170193>, line 46
     41 df = spark.read.option("header", True).option("inferSchema", True).csv(
     42     f"wasbs://mmix-blob-storage@mmixstorage.blob.core.windows.net/{input_path}"
     43 )
     45 # Convert date column to DateType
---> 46 df = df.withColumn(date_col, F.to_timestamp(F.col(date_col), "M/d/yyyy H:mm"))
     48 # ====================
     49 # 4. Apply adstock + saturation
     50 # ====================
     51 for col, trans in transformations.items():

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50    